### To do:
Percent of sentences with however

Maximum sentence

Percent that end with unclear

In [1]:
import numpy as np
import pdb
from Mechanisms_bot.src import pubmed_io
import Mechanisms_bot.tweet_mechanism as tweet_mechanism
from Mechanisms_bot.src import abstract_parser 
import importlib
import pickle

In [3]:
pubmed_ids = pubmed_io.get_pubmed_ids(95000)
len(pubmed_ids)

95000

In [37]:
importlib.reload(pubmed_io)
pubmed_abstracts = pubmed_io.get_pubmed_abstracts( pubmed_ids[:1000])
len(pubmed_abstracts)

998

In [33]:
[x for x in pubmed_abstracts if not x]

[]

In [35]:
str([1,None,3])

'[1, None, 3]'

Pubmed returns an XML. It is structured PubmedArticleList / PubmedArticle / MedlineCitation. MedlineCitation has children: PMID, and Article

Article has children: Journal, ArticleTitle, and Abstract

Abstract has child: AbstractText

In [56]:
good_sent = list(map(abstract_parser.get_mech_sent, pubmed_abstracts, pubmed_ids) )
good_sent = [x for x in good_sent if x['mech_sent']]
len(good_sent)

27

### All of the above in one function

In [2]:
tweet_mechanism.get_recent_mech_sentences(90000)

INFO:Mechanisms_bot.tweet_mechanism:Getting recent papers


KeyboardInterrupt: 

# Tweeting!

In [10]:
importlib.reload(tweet_mechanism)
new_sent, tweeted_sentences = tweet_mechanism.main();

INFO:Mechanisms_bot.tweet_mechanism:Getting recent papers
INFO:Mechanisms_bot.tweet_mechanism:Got 100 recent PubMed IDs
INFO:Mechanisms_bot.tweet_mechanism:Getting abstracts
INFO:Mechanisms_bot.tweet_mechanism:Downloaded 100 abstracts
INFO:Mechanisms_bot.tweet_mechanism:Found 22 mechanisms sentences

